In [1]:
%run "template.ipynb"

In [2]:
from sqlmodel import SQLModel, Relationship, Field as SqlField
from typing import TYPE_CHECKING
from sqlalchemy import Column, DateTime, text
from datetime import datetime
from uuid import UUID, uuid4


class Article(SQLModel, table=True):
    __tablename__ = "articles"
    __table_args__ = {"extend_existing": True}
    id: UUID = SqlField(default_factory=uuid4, primary_key=True)
    text: str

    created_at: datetime = SqlField(
        sa_column=Column(
            DateTime(timezone=True),
            nullable=False,
            server_default=text("CURRENT_TIMESTAMP"),
        )
    )
    updated_at: datetime = SqlField(
        sa_column=Column(
            DateTime(timezone=True),
            nullable=False,
            server_default=text("CURRENT_TIMESTAMP"),
            onupdate=text("CURRENT_TIMESTAMP"),
        )
    )

    plant_id: UUID = SqlField(foreign_key="plants.id")


In [3]:
df = pd.read_csv("articles.csv", index_col=0)

In [4]:
df

,text
8f49ec5e-3bf4-4b4b-bcc7-2af3a8c217f7,## Матграсс: Уход за домашним растением\n\n**М...
00c01774-86a6-46f1-b4cb-b595a68576cd,## **Комнатное растение: Плантейн**\n\n**Плант...
040506ad-12e1-436d-a2dc-4d08e7eba2d5,## Cinquefoil: Уход за домашними цветами\n\n**...
06f2945a-47f6-4e4b-949b-31b32cf9ab1b,## Cow-parsnip: Уход за домашним растением\n\n...
082c6d57-1949-4336-ae11-8e73f88f3a60,## Mountain Geranium\n\n**Mountain geranium** ...
0a54b55b-6175-4750-8b9b-f132d8b1869c,## Quickbeam: Уход за домашней растительностью...
0f0394a9-138e-4154-a914-5e4d0f98277d,## Meadow Cress: Уход и Особенности Выведения\...
11b25fcd-3a66-4021-a1d1-677df8bf1477,## Нарцисс и его уход в домашних условиях\n\n*...
147bbcbb-b9df-45b2-9267-14087b957b26,## Гарденинские ленточки\n\n**Гарденинские лен...
14e0ec3e-4ac8-479b-b3b9-946df44e2c00,## Tufted Hairgrass: Уход и Особенности Выведе...


In [ ]:
from sqlalchemy import text, insert

async with session_maker() as session:
    for row in df.iterrows():
        stmt = insert(Article).values(plant_id=UUID(row[0]), text=row[1]["text"])
        await session.exec(stmt)

    await session.commit()